# Damage Calculation

This notebook shows a general calculation stream for a nominal damage calculation.

1. Define load cycles
2. Define the material parameter
3. Select the damage calculation method (Miner elementary, Miner-Haibach, ...)
4. Calculate the damage for every load level and the damage sum


In [1]:
import numpy as np
import pandas as pd
import pylife.strength.fatigue
import pylife.vmap
import pylife.stress.equistress
import plotly.express as px
import pylife.mesh.meshsignal
import pylife.mesh.hotspot
import pylife.vmap
import pyvista as pv

### 1. Define load cycles


In [2]:
load_cycles = pd.DataFrame(
    {
        "range": [100, 50, 75, 25],
        "mean": 0.0,
        "cycles": [1e3, 5e3, 10e3, 25e3],
    },
    index=pd.Index([1, 2, 3, 4], name="load_event")
)

In [3]:
load_cycles

,range,mean,cycles
load_event,,,
1,100,0.0,1000.0
2,50,0.0,5000.0
3,75,0.0,10000.0
4,25,0.0,25000.0


In [4]:
vm_mesh = pylife.vmap.VMAPImport("two_notched_beam.vmap")
mesh = (vm_mesh.make_mesh('beam', 'STATE-1')
        .join_coordinates()
        .join_variable('STRESS_CAUCHY')
        .to_frame())

In [5]:
mesh['mises'] = mesh.equistress.mises() / 1000

In [6]:
mesh

x          y     z          S11         S22  \
element_id node_id                                                        
1          352     -43.969048  23.989227   0.0  1229.652344  -76.191574   
           359     -42.955109  24.005196   0.0  1229.652344  -76.191574   
           215     -43.000000  25.000000   0.0  1229.652344  -76.191574   
           216     -44.000000  25.000000   0.0  1229.652344  -76.191574   
           1271    -43.969048  23.989227   1.0  1229.652344  -76.191574   
...                       ...        ...   ...          ...         ...   
8340       9186    -35.655140  17.474123   9.0  1054.807251 -448.547363   
           10106   -36.133118  17.862680  10.0  1054.807251 -448.547363   
           9903    -36.538712  17.142405  10.0  1054.807251 -448.547363   
           9796    -35.728638  16.920214  10.0  1054.807251 -448.547363   
           10105   -35.655140  17.474123  10.0  1054.807251 -448.547363   

                          S33         S12        S13        S23     mises  
element_id node_id                                                         
1          352     -54.879715  -28.192242   4.629902  -3.550738  1.296279  
           359     -54.879715  -28.192242   4.629902  -3.550738  1.296279  
           215     -54.879715  -28.192242   4.629902  -3.550738  1.296279  
           216     -54.879715  -28.192242   4.629902  -3.550738  1.296279  
           1271    -54.879715  -28.192242   4.629902  -3.550738  1.296279  
...                       ...         ...        ...        ...       ...  
8340       9186     -0.840952 -402.977814 -11.581802  10.506829  1.508430  
           10106    -0.840952 -402.977814 -11.581802  10.506829  1.508430  
           9903     -0.840952 -402.977814 -11.581802  10.506829  1.508430  
           9796     -0.840952 -402.977814 -11.581802  10.506829  1.508430  
           10105    -0.840952 -402.977814 -11.581802  10.506829  1.508430  

[66720 rows x 10 columns]

In [7]:
grid = pv.UnstructuredGrid(*mesh.mesh.vtk_data())
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars=mesh.groupby('element_id')['mises'].mean().to_numpy(), show_edges=True)
plotter.add_scalar_bar()
plotter.show()

Widget(value='<iframe src="http://localhost:49382/index.html?ui=P_0x21a5e963640_0&reconnect=auto" class="pyvis…

In [8]:
scaled_collective = load_cycles.load_collective.scale(mesh['mises'])
# scaled_collective.amplitude

## 2. Define the material parameter


In [9]:
k_1 = 8
mat = pd.Series(
    {
        "k_1": k_1,
        'k_2' : 20,
        "ND": 1.0e6,
        "SD": 400.0,
        "TN": 12.0,
        "TS": 1.1,
    }
)
# display(mat)

In [10]:
damage = mat.fatigue.miner_original().damage(scaled_collective)
# damage

In [11]:
grouped_damage = damage.groupby(["load_event", "element_id"]).mean().groupby("element_id").sum()
grouped_damage[grouped_damage > 0.0]

element_id
38      0.001666
872     0.003767
1191    0.001882
1706    0.003035
2025    0.001709
2540    0.003198
2859    0.001751
3374    0.003074
3693    0.001728
4208    0.003074
4527    0.001728
5042    0.003198
5361    0.001751
5876    0.003035
6195    0.001709
6710    0.003767
7029    0.001882
7544    0.001666
Name: damage, dtype: float64

In [12]:
mesh_result = mesh.join(grouped_damage)
grid = pv.UnstructuredGrid(*mesh_result.mesh.vtk_data())
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars=grouped_damage.to_numpy(), show_edges=True)
plotter.add_scalar_bar()
plotter.show()

Widget(value='<iframe src="http://localhost:49382/index.html?ui=P_0x21a5e992f50_1&reconnect=auto" class="pyvis…